# Sentimental Analysis of YouTube Videos

#### Sentiment Analysis is one of the Natural Language Processing techniques, which can be used to determine the sensibility behind the texts, i.e. tweets, movie reviews, youtube comments, any incoming message, etc. 

Here we will perform sentiment analysis on youtube video’s comments. To carry out the sentiment analysis on any youtube video, 
the first thing we need is the comments on that video which can be extracted here below.
With the help of this sentiment analysis of comments, the user can get to know about the community acceptance of its channel/video based on that one can maintain their content quality.
Another use case of the same is analyzing the trending video as many times, there are videos with more views and likes on trending
 but if you will use sentiment analysis you will easily be able to find the most useful video of a particular channel, celebrity, category, etc.

### 1. Import necessary package and library

In [69]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [70]:
pip install -U textblob

Requirement already up-to-date: textblob in c:\programdata\anaconda3\lib\site-packages (0.17.1)
Note: you may need to restart the kernel to use updated packages.


In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
from textblob import TextBlob
pd.get_option("display.max_columns",None)
pd.get_option("display.max_rows",None)

60

### 2. Reading files

In [72]:
videos = pd.read_csv('USvideos.csv',encoding='utf8',error_bad_lines = False)
comm = pd.read_csv('UScomments.csv',encoding='utf8',error_bad_lines = False)

b'Skipping line 2401: expected 11 fields, saw 21\nSkipping line 2800: expected 11 fields, saw 21\n'
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'


### 3. Inspecting the dataframe

In [73]:
videos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [74]:
comm.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


##### 3.1 Making the BOB classifier and using it to test the sentiments of the sentence


In [ ]:
for i in comm.comment_text.values:
    try:
        analysis =TextBlob(i)
        pol.append(analysis.sentiment.polarity)
    
    except:
        pol.append(0)

In [ ]:
print(pol==0)

##### 3.2 Converting the continuous variable to categorical one

In [ ]:
comm['pol']=pol
comm['pol'][comm.pol==0]= 0
comm['pol'][comm.pol>0]= 1
comm['pol'][comm.pol<0]= -1

---------Reading positive comments

In [ ]:
positive = comm[comm.pol==1]
positive.head()

In [ ]:
k  = (' '.join(positive['comment_text']))
wordcloud = WordCloud(width = 1000,height=500).generate(k)
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')

----------Reading negative comments

In [ ]:
negative = comm[comm.pol==-1]
k=(' '.join(negative['comment_text']))
wordcloud = WordCloud(width=1000,height=500).generate(k)
plt.figure(figsize=(15,5,))
plt.imshow(wordcloud)
plt.axis('off')

##### 3.3 Lets check number of types of comments

In [ ]:
comm['pol'].replace({1:'Positive',0:'Neutral',-1:'Negative'}).value_counts().plot(kind='bar',figsize=(7,4));
plt.title('Number of comments');
plt.xlabel('Comments_type');
plt.ylabel('Number');

##### 3.4 Generating dataframe with unique id and their comments

In [ ]:
id = []
pos_comm = []
neg_comm = []
neutral_comm = []
for i in set(comm.video_id):
    id.append(i)
    try:
        pos_comm.append(comm[comm.video_id==i].pol.value_counts()[1])
    except:
        pos_comm.append(0)
    try:
        neg_comm.append(comm[comm.video_id==i].pol.value_counts()[-1])
    except:
        neg_comm.append(0)
    try:
        neutral_comm.append(comm[comm.video_id==i].pol.value_counts()[0])
    except:
        neutral_comm.append(0)

In [ ]:
df_unique = pd.DataFrame(id)
df_unique.columns=['id']
df_unique['pos_comm'] = pos_comm
df_unique['neg_comm'] = neg_comm
df_unique['neutral_comm'] = neutral_comm
df_unique['total_comments'] = df_unique['pos_comm'] + df_unique['neg_comm']+df_unique['neutral_comm']
df_unique.head()

---------------Saving for further analysis

In [ ]:
df_unique.to_csv('unique.csv',index=False)

### 4. Exploratory Data Analysis

In [ ]:
videos.head()

In [ ]:
comm.head()

-------Date is of range 13.09 to 3.10(3 weeks)

In [ ]:
videos.date.value_counts().sort_index()

-------Trending videos for a week

In [ ]:
print(videos.video_id.value_counts()[:13])
more_trending = videos.video_id.value_counts()[:13].index

In [ ]:
videos[videos.video_id=='dInwVhRtN4E']

If we see this particular id started trending from 20.09 to 26.09, for a week it was in most trending list

In [ ]:
for i in more_trending:
    info = videos[videos.video_id==i][['video_id','title','channel_title','views','likes','dislikes',
                                       'comment_total','date']]
    print(info)
    print("***********************")

If we compare the views for each video it gradually increases for a week as it is expected . It is clear why they are more trending for a week by looking number of likes

In [ ]:
tags = videos['tags'].map(lambda k: k.lower().split('|')).values
k = (' '.join(videos['tags']))
wordcloud = WordCloud(width=1000,height=500).generate((' '.join(k.lower().split('|'))))
plt.figure(figsize=(15,5))
plt.imshow(wordcloud)
plt.axis('off')

--------iphone,makeup,tutorial,music,video,beauty,funny are some most common tags on these videos

In [ ]:
videos.columns

In [ ]:
df1 = pd.DataFrame(videos.channel_title.value_counts())
df1.columns=['times channel got trending']
df1.head().sort_index(0)

First We Feast, NFL, The Late Late Show with James Corden, The Tonight Show Starring Jimmy Fallon and Vox are too five channel with got trending 21 times.

In [ ]:
df_channel = pd.DataFrame(videos.groupby(by=['channel_title'])['views'].mean()).sort_values(by='views',
                                                                                        ascending=False)
df_channel.head(10).plot(kind='bar')
plt.title('Most viwed channels');

---------ZaynVEVO leads to be most viwed channel among all !

In [ ]:
df_channel = pd.DataFrame(videos.groupby(by=['channel_title'])['likes'].mean()).sort_values(by='likes',
                                                                                        ascending=False)
df_channel.head(10).plot(kind='bar')
plt.title('Most viwed channels');

----------ZaynVEVO leads to be most liked channel among all !

In [ ]:
videos['likes_per_view'] = videos['likes']/videos['views']
df_channel = pd.DataFrame(videos.groupby(by=['channel_title'])['likes_per_view'].mean()).sort_values(by='likes_per_view',ascending=False)
df_channel.head(10).plot(kind='bar');
plt.title('Most liked channels');

In [ ]:
videos['dislikes_per_view'] = videos['dislikes']/videos['views']
df_channel = pd.DataFrame(videos.groupby(by=['channel_title'])['dislikes_per_view'].mean()).sort_values(by='dislikes_per_view',ascending=False)
df_channel.head(10).plot(kind='bar');
plt.title('Most disliked channels');

-------CBS Sunday Morning is the most disliked channel

##### 4.1 Inferences

In [ ]:
unique = pd.read_csv('unique.csv')

In [ ]:
unique.sort_values(by = 'pos_comm',ascending = False).head()

In [ ]:
videos[videos.video_id == 'eERPlIdPJtI'].title[225]

Inference---- Mostly 'Weight Update: 6 weeks Post Surgery! 93 pounds!' have very large number of positive reviews


In [ ]:
sns.barplot(data=unique.sort_values(by='pos_comm',ascending=False).head(10),x ='id',y='pos_comm')
plt.xticks(rotation=45)
plt.figure(figsize=(5,4))

In [ ]:
sns.barplot(data=unique.sort_values(by='neg_comm',ascending=False).head(10),x ='id',y='neg_comm')
plt.xticks(rotation=45)
plt.figure(figsize=(5,4))

In [ ]:
sns.barplot(data=unique.sort_values(by='neutral_comm',ascending=False).head(10),x ='id',y='neutral_comm')
plt.xticks(rotation=45)
plt.figure(figsize=(5,4))

Number of likes have strong relationship with views

In [ ]:
sns.regplot(data=videos,x='views',y='likes')
plt.title("Regression plot for likes and views");

Check relation between views and dislikes

In [ ]:
sns.regplot(data=videos,x='dislikes',y='views')
plt.title("Regression plot for dislikes and views");

### 5 Correlation matrix -Evidence of above anlaysis

In [ ]:
df_corr = videos[['views','likes','dislikes']]
plt.subplots(figsize=(8,5))  
sns.heatmap(df_corr.corr(),annot=True, linewidths=1,diag_kind="kde")
#plt.figsize(5,5)
#fig, ax =        # Sample figsize in inches
#sns.heatmap(df1.iloc[:, 1:6:], annot=True, linewidths=.5, ax=ax)

With above Co-relation matrix we can see number of likes and views are totaly co related quantity having .83 relation